In [ ]:
# import the necesssary packages
%matplotlib inline
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt
import torchvision.utils
import numpy as np
import random
import os
from PIL import Image
import torch
from torch.autograd import Variable
import PIL.ImageOps    
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from prettytable import PrettyTable

In [ ]:
def imshow(img,text=None,should_save=False):
    npimg = img.numpy()
    plt.axis("off")
    if text:
        plt.text(75, 8, text, style='italic',fontweight='bold',
            bbox={'facecolor':'white', 'alpha':0.8, 'pad':10})
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [ ]:
# labelling similar and dissimilar image pairs
class SiameseDataset(Dataset):
    def __init__(self,root_dir,imageFolderDataset, transform=None, should_invert=True):
        self.root_dir=root_dir
        self.imageFolderDataset=imageFolderDataset
        self.transform=transform
        self.should_invert=should_invert
        
    def __getitem__(self,index):
        should_get_same_class=random.randint(0,1)
        
        img1_tuple=random.choice(self.imageFolderDataset.imgs)
        if should_get_same_class:
            while True:
                img2_tuple=random.choice(self.imageFolderDataset.imgs)
                if img1_tuple[1]==img2_tuple[1]:
                    break
        else:
            while True:
                img2_tuple=random.choice(self.imageFolderDataset.imgs)
                if img1_tuple[1]!=img2_tuple[1]:
                    break
        
        img1=Image.open(img1_tuple[0])
        img2=Image.open(img2_tuple[0])
        
        if self.should_invert:
            img1=PIL.ImageOps.invert(img1)
            img2=PIL.ImageOps.invert(img2)
        
        if self.transform is not None:
            img1=self.transform(img1)
            img2=self.transform(img2)
            
        return img1, img2, torch.from_numpy(np.array([(img1_tuple[1]!=img2_tuple[1])], dtype=np.float32))
    
    def __len__(self):
        return len(self.imageFolderDataset.imgs)

In [ ]:
# define image size, no of epochs and batch size
img_size=105 
epochs=50
batchsize=64

In [ ]:
train_dir='./data/Omniglot/train/'
train_imagefolder=dset.ImageFolder(train_dir)

In [ ]:
train_dataset=SiameseDataset(root_dir=train_dir, imageFolderDataset=train_imagefolder,
                             transform=transforms.Compose([transforms.Resize((img_size,img_size)),
                                                           transforms.RandomHorizontalFlip(),
                                                           transforms.RandomResizedCrop(img_size, scale=(0.8,1.0)),
                                                           transforms.ToTensor()
                                                          ]),
                             should_invert=False)

train_dataloader=DataLoader(train_dataset, batch_size=batchsize, num_workers=0, shuffle=True)

print("Train dataset has {} images\n".format(len(train_dataset)))
print("Number of batches in train dataset with batch size of {} is {}".format(train_dataloader.batch_size,len(train_dataloader)))

In [ ]:
# visualize how images in a batch are labelled as similar and dissimilar
visualize_dataloader = DataLoader(train_dataset,
                        shuffle=True,
                        num_workers=0,
                        batch_size=8)
dataiter = iter(visualize_dataloader)


example_batch = next(dataiter)
concatenated = torch.cat((example_batch[0],example_batch[1]),0)
imshow(torchvision.utils.make_grid(concatenated))
print(example_batch[2].numpy()) #same == 0  #different == 1

In [ ]:
#Koch et al.   
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 64, 10),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 7),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Conv2d(128, 128, 4),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Conv2d(128, 256, 4),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
        )
        
        self.dropout=nn.Dropout(p=0.5) 
        
        self.fc1=nn.Sequential(
            nn.Linear(9216, 4096),
            nn.Sigmoid())
        
        self.out=nn.Linear(4096,1)
        
    def forward_once(self,inp):
        inp=self.conv(inp)
        inp=inp.view(inp.size()[0], -1)
        inp = self.dropout(inp) #
        inp=self.fc1(inp)
        return inp
        
    def forward(self, inp1, inp2):
        out1=self.forward_once(inp1)
        out2=self.forward_once(inp2)
        return out1,out2

In [ ]:
# no of parameters in the network
def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params
    
count_parameters(SiameseNetwork())

In [ ]:
# if CUDA is available, set device to CUDA
if torch.cuda.is_available():
    print(f"Number of CUDA devices available :  {torch.cuda.device_count()}")
    print(f"GPU Name : {torch.cuda.get_device_name(0)}")
else:
    print("No CUDA devices found")
    
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
# define contrastive loss
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


        return loss_contrastive

In [ ]:
# set loss criterion and optimizer
net=SiameseNetwork().cuda()
criterion=ContrastiveLoss()
optimizer=optim.Adam(net.parameters(), lr=0.001)

In [ ]:
valid_dir='./data/Omniglot/valid/'
valid_imagefolder=dset.ImageFolder(valid_dir)

In [ ]:
valid_dataset=SiameseDataset(root_dir=valid_dir, imageFolderDataset=valid_imagefolder,
                             transform=transforms.Compose([transforms.Resize((img_size,img_size)),
                                                           transforms.RandomHorizontalFlip(),
                                                           transforms.RandomResizedCrop(img_size, scale=(0.8,1.0)),
                                                           transforms.ToTensor()
                                                          ]),
                             should_invert=False)

valid_dataloader=DataLoader(valid_dataset, batch_size=batchsize, num_workers=0, shuffle=True)

print("Validation dataset has {} images\n".format(len(valid_dataset)))
print("Number of batches in validation dataset with batch size of {} is {}".format(valid_dataloader.batch_size,len(valid_dataloader)))

In [ ]:
def train(net, train_dataloader, valid_dataloader, epochs, criterion):
    train_loss=[] # training loss for every epoch
    valid_loss=[] # validation loss for every epoch
    sum_train_loss=0.0 # sum of training losses for every epoch
    sum_valid_loss=0.0 # sum of validation losses for every epoch
    
    for epoch in range(1, epochs+1):
        train_epoch_loss=0.0
        net.train()
        for i, data in enumerate(train_dataloader,0):
            img1, img2, label = data
            img1, img2, label = img1.cuda(), img2.cuda(), label.cuda()
            
            label = label.float()
            
            output1, output2 = net(img1, img2)
            loss = criterion(output1, output2, label)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_epoch_loss = train_epoch_loss + ((1/(i+1)) * (loss.item() - train_epoch_loss))
            
        train_loss.append(train_epoch_loss)
        sum_train_loss+=train_epoch_loss
        
        valid_epoch_loss=0.0
        with torch.no_grad():
            net.eval()
            for i, data in enumerate(valid_dataloader,0):
                img1, img2, label = data
                img1, img2, label = img1.cuda(), img2.cuda(), label.cuda()
                
                output1, output2 = net(img1, img2)
                loss = criterion(output1, output2, label)
                
                valid_epoch_loss = valid_epoch_loss + ((1/(i+1)) * (loss.item() - valid_epoch_loss))
                
        valid_loss.append(valid_epoch_loss)
        sum_valid_loss+=valid_epoch_loss
        
        print("Epoch {}/{}\n Train loss : {} \t Valid loss {}\n"
             .format(epoch, epochs, train_epoch_loss, valid_epoch_loss))
        
    print("Average training loss after {} epochs : {}".format(epochs, sum_train_loss/epochs))
    print("Average validation loss after {} epochs : {}".format(epochs, sum_valid_loss/epochs))
    
    return train_loss, valid_loss

In [ ]:
# training and validation
train_losses, valid_losses = train(net, train_dataloader, valid_dataloader, epochs, criterion)

In [ ]:
# plot training loss and validation loss
plt.xlabel('epochs')
plt.ylabel('loss')
plt.plot(train_losses, label="Train loss")
plt.plot(valid_losses, label="Validation loss")
plt.legend(bbox_to_anchor=(1.1,1.0), loc='upper left')
plt.savefig('plot.png',dpi=300,bbox_inches='tight')
plt.show()

In [ ]:
test_dir='./data/Omniglot/test/'
test_imagefolder=dset.ImageFolder(test_dir)

In [ ]:
test_dataset=SiameseDataset(root_dir=test_dir, imageFolderDataset=test_imagefolder,
                             transform=transforms.Compose([transforms.Resize((img_size,img_size)),
                                                           transforms.RandomHorizontalFlip(),
                                                           transforms.RandomResizedCrop(img_size, scale=(0.8,1.0)),
                                                           transforms.ToTensor()
                                                          ]),
                             should_invert=False)

test_dataloader=DataLoader(test_dataset, batch_size=1, num_workers=0, shuffle=True)

print("Test dataset has {} images\n".format(len(test_dataset)))
print("Number of batches in test dataset with batch size of {} is {}".format(test_dataloader.batch_size,len(test_dataloader)))

In [ ]:
def eval(net, test_dataloader):
    with torch.no_grad():
        net.eval()
        count=100
        correct=0
        correct_count=0
        dataiter = iter(test_dataloader)
        print("Testing...")
        for i in range(count):
            img1,img2,label=next(dataiter)
            cat=torch.cat((img1, img2),0)
            output1, output2 = net(Variable(img1).cuda(), Variable(img2).cuda()) 
            euclidean_distance = F.pairwise_distance(output1, output2) 
            prediction = torch.sigmoid(euclidean_distance)
            total = label.size(0)
            
            # check if prediction and actual label are same
            for j in range(label.size(0)):
                if (prediction[j]>0.5) and (label[j]==1):
                    correct+=1
                elif (prediction[j]<0.5) and (label[j]==0):
                    correct+=1
                
            correct_count+=correct/total
            correct=0
            imshow(torchvision.utils.make_grid(cat),'Pred : {:.2f} Label : {}'.format(prediction.item(),label.item()))      
    return correct_count, count, (correct_count/count)*100

In [ ]:
# testing
acc=eval(net, test_dataloader)
print('{} correct predictions out of {}\nAccuracy : {:.2f}'.format(acc[0],acc[1], acc[2]))